# I. $LU$ - разложение квадратной матрицы

Рассмотрим наивную реализацию LU - разложения.  

Заметим, что мы используем массивы `numpy` для представления матриц. [Не используйте 'np.matrix'].

In [1]:
import numpy as np

def diy_lu(a):
    """Создает LU - разложение матрицы `a`.
    
    Наивное LU - разложение: работает столбец за столбцом, накапливает элементарные треугольные матрицы.
    Без выбора главного элемента.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Теперь сгенерируем матрицу полного ранга и протестируем наивное разложение.
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Настройка вывода чисел с плавающей точкой для большей ясности
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Быстрый тест на адекватность: L @ U должна быть равна изначальной матрице с точностью до ошибок округления.
print(L@u - a)

[[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 1.     1.455  1.     0.     0.     0.   ]
 [ 1.     1.714  1.742  1.     0.     0.   ]
 [ 1.     1.882  2.276  2.039  1.     0.   ]
 [ 1.     2.     2.671  2.944  2.354  1.   ]] 

[[  3.000e+00   3.000e+00   3.000e+00   3.000e+00   3.000e+00   3.000e+00]
 [  0.000e+00  -1.125e+00  -1.636e+00  -1.929e+00  -2.118e+00  -2.250e+00]
 [  0.000e+00   0.000e+00   2.625e-01   4.574e-01   5.975e-01   7.013e-01]
 [  0.000e+00   2.220e-16   0.000e+00  -2.197e-02  -4.480e-02  -6.469e-02]
 [  0.000e+00  -4.528e-16   0.000e+00   6.939e-18   8.080e-04   1.902e-03]
 [  0.000e+00   4.123e-16   0.000e+00  -1.634e-17   0.000e+00  -1.585e-05]] 

[[  0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   0.000e+00   0.000e+00   2.220e-16  -1.110e-16  -1.665e-16]
 [  0.000e+00   0.000e+00   2.220e-16  -5.551e-17

# II. Необходимость выбора главного элемента

Давайте немного подправим матрицу, изменив в ней один элемент:

In [6]:
a1 = a.copy()
a1[1, 1] = 3

Результирующая матрица имеет полный ранг, но наивное LU - разложение не работает.

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u)

[[ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]] [[ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]]


### Тест II.1

Для того, чтобы наивное LU - разложение работало необходимо чтобы все лидирующие миноры матрицы были отличны от нуля. Проверьте, выполнено ли это требование для двух матриц `a` и `a1`.

(20% оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...

### Тест II.2

Модифицируйте алгоритм `diy_lu`, чтобы осуществлять выбор главного элемента в столбцах. Для контроля выбора можете использовать матрицу перестановок или массив замен.

(40% оценки)

Напишите функию, воссоздающую изначальную матрицу из разложения. Протестируйте свой алгоритм на матрицах `a` и `a1`.

(40% оценки)

In [ ]:
# ... ENTER YOUR CODE HERE ...